In [1]:
import json

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from gnn_mp_model.pipelines.data_featurization.utils import from_smiles
from lightning.pytorch.callbacks import (
    EarlyStopping,
    LearningRateMonitor,
    ModelCheckpoint,
)
from lightning.pytorch.loggers import CSVLogger
from mango import Tuner, scheduler
from pytorch_lightning.utilities.model_summary import ModelSummary
from rdkit import Chem
from rdkit.Chem import AllChem, Draw
from rdkit.Chem.Draw import IPythonConsole
from sklearn.metrics import r2_score as sklearn_r2
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from torch.nn import BatchNorm1d, HuberLoss, L1Loss, Linear, ModuleList, MSELoss
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.loader import DataLoader
from torch_geometric.nn import (
    GATv2Conv,
    GCNConv,
    GINConv,
    GraphNorm,
    PNAConv,
    SAGPooling,
    SetTransformerAggregation,
    global_add_pool,
    global_max_pool,
    global_mean_pool,
)
from torcheval.metrics.functional import r2_score
from twinning import twin


No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'dgl'
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (c:\Users\01121272\Desktop\Projects\gnn-mp-model\.venv\Lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading some PyTorch models, missing a dependency. No module named 'tensorflow'
c:\Users\01121272\Desktop\Projects\gnn-mp-model\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("data/zinc.csv")

In [3]:
df

,smiles,logP,qed,SAS
0,CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1\n,5.05060,0.702012,2.084095
1,C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1\n,3.11370,0.928975,3.432004
2,N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)...,4.96778,0.599682,2.470633
3,CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c...,4.00022,0.690944,2.822753
4,N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#...,3.60956,0.789027,4.035182
...,...,...,...,...
249450,CC1(C)CC[C@H](CNC(=O)Cn2ncc3ccccc3c2=O)c2ccccc...,3.36790,0.745901,2.900726
249451,Cn1ccnc1C(=O)c1ccc(NC(=O)C2CCN(C(=O)C(C)(C)C)C...,2.87430,0.799426,2.326627
249452,Cc1ccc(NC(=O)C(=O)N(C)Cc2ccccc2)c(C)c1\n,2.90054,0.878086,1.840642
249453,Cc1cc(C(=O)Nc2ccc(OCC(N)=O)cc2)c(C)n1C1CC1\n,2.55624,0.852917,2.024638


In [4]:
df.drop(labels=["qed", "SAS"], axis=1, inplace=True)

In [7]:
df.rename(columns={"logP": "MP"}, inplace=True)

In [8]:
df

,smiles,MP
0,CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1\n,5.05060
1,C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1\n,3.11370
2,N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)...,4.96778
3,CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c...,4.00022
4,N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#...,3.60956
...,...,...
249450,CC1(C)CC[C@H](CNC(=O)Cn2ncc3ccccc3c2=O)c2ccccc...,3.36790
249451,Cn1ccnc1C(=O)c1ccc(NC(=O)C2CCN(C(=O)C(C)(C)C)C...,2.87430
249452,Cc1ccc(NC(=O)C(=O)N(C)Cc2ccccc2)c(C)c1\n,2.90054
249453,Cc1cc(C(=O)Nc2ccc(OCC(N)=O)cc2)c(C)n1C1CC1\n,2.55624


In [5]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
def _generate_graph_list(df: pd.DataFrame):
    data_list = []
    for _, row in df.iterrows():
        smiles = row["smiles"]
        label = row["logP"]
        graph = from_smiles(smiles)
        graph.y = label
        data_list.append(graph)
    return data_list

def generate_graph_loader(df: pd.DataFrame):
    data_list = _generate_graph_list(df)
    graph_loader = DataLoader(
        data_list,
        batch_size=128,
        shuffle=True,
        drop_last=True,
    )
    return graph_loader

In [ ]:
zinc_dataloader = generate_graph_loader(df.sample(50000))

In [ ]:
%load_ext kedro.ipython

In [ ]:
train_dataloader = catalog.load("sys_train_dataloader")
test_dataloader = catalog.load("sys_test_dataloader")

In [ ]:
# Plot data split

# fig, axs = plt.subplots(1,2,sharey=True, tight_layout=True)
# n_bins = 30
# axs[0].hist(df_strain["MP"], bins=n_bins, color="b")
# axs[0].hist(df_stest["MP"], bins=n_bins, color="r")
# axs[1].hist(df_train["MP"], bins=n_bins, color="b")
# axs[1].hist(df_test["MP"], bins=n_bins, color="r")


In [ ]:
# Compute the maximum in-degree in the training data.
from torch_geometric.utils import degree

max_degree = -1
for batch in train_dataloader:
    for i in range(batch.batch_size):
        data = batch[i]
        d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
        max_degree = max(max_degree, int(d.max()))

# Compute the in-degree histogram tensor
deg = torch.zeros(max_degree + 1, dtype=torch.long)
for batch in train_dataloader:
    for i in range(batch.batch_size):
        data = batch[i]
        d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
        deg += torch.bincount(d, minlength=deg.numel())

In [ ]:
class GNN(torch.nn.Module):
    def __init__(self, model_type: str, num_layers: int, hidden_size: int, dense_size: int, pool_rate: float, normalization: bool):  # noqa: PLR0913
        # Loading params
        super().__init__()
        self.model_type = model_type
        self.num_layers = num_layers
        self.normalization = normalization
        node_dim = 9
        edge_dim = 4
        # PNA
        self.pan_list = ModuleList([])
        self.gn_list = ModuleList([])
        self.aggr_list = ModuleList([])
        self.pool_list = ModuleList([])
        # Initial layers
        self.pan = PNAConv(
                in_channels=node_dim,
                out_channels = hidden_size,
                edge_dim=edge_dim,
                aggregators=["mean", "min", "max", "std"],
                scalers=["identity", "amplification", "attenuation"],
                deg=deg,
                towers=1)
        self.pool = SAGPooling(hidden_size, pool_rate)
        self.gn = GraphNorm(hidden_size)
        self.aggr = SetTransformerAggregation(hidden_size)
        for _ in range(self.num_layers):
            self.pan_list.append(PNAConv(
                in_channels=hidden_size,
                out_channels = hidden_size,
                edge_dim=edge_dim,
                aggregators=["mean", "min", "max", "std"],
                scalers=["identity", "amplification", "attenuation"],
                deg=deg,
                towers=1))
            self.pool_list.append(SAGPooling(hidden_size, pool_rate))
            self.gn_list.append(GraphNorm(hidden_size))
            self.aggr_list.append(SetTransformerAggregation(hidden_size))

        # Linear layers
        self.linear1 = Linear(hidden_size, dense_size)
        self.linear2 = Linear(dense_size, int(dense_size / 2))
        self.linear3 = Linear(int(dense_size / 2), 1)

    def forward(self, x, edge_attr, edge_index, batch_index):
        # PNA
        global_representation = []
        if self.model_type == "PNA":
            x = self.pan(x, edge_index, edge_attr)
            x, edge_index, edge_attr, batch_index, _, _ = self.pool(
            x, edge_index, edge_attr, batch_index
        )
            if self.normalization is True:
                x = self.gn(x, batch_index)
            global_representation.append(self.aggr(x, batch_index))
            for i in range(self.num_layers):
                x = self.pan_list[i](x, edge_index, edge_attr)
                x, edge_index, edge_attr, batch_index, _, _ = self.pool_list[i](
            x, edge_index, edge_attr, batch_index
        )
                if self.normalization is True:
                    x = self.gn_list[i](x, batch_index)
                global_representation.append(self.aggr_list[i](x, batch_index))
        # Output block
        x = sum(global_representation)
        x = torch.relu(self.linear1(x))
        x = F.dropout(x, p=0.2, training=self.training)
        x = torch.relu(self.linear2(x))
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.linear3(x)
        return x

In [ ]:
# batch=next(iter(train_dataloader))

# model = GNN("PNA", num_layers=2)
# model(batch.x, batch.edge_attr, batch.edge_index, batch.batch)

In [ ]:
import lightning as L


class GNN_L(L.LightningModule):
    def __init__(self, model, lr: float, weight_decay: float, gamma: float):
        super().__init__()
        self.model = model
        self.lr = lr
        self.weight_decay = weight_decay
        self.loss_fn = MSELoss()
        self.gamma = gamma

    def forward(self, x, edge_attr, edge_index, batch_index):
        return self.model(
            x.float(), edge_attr.float(), edge_index, batch_index
        )

    def training_step(self, batch, batch_idx):
        preds = self(
            batch.x.float(), batch.edge_attr.float(), batch.edge_index, batch.batch
        ).squeeze()
        target = batch.y.float()
        loss = self.loss_fn(preds, target)
        r2 = r2_score(preds, target)
        rmse = root_mean_squared_error(target.numpy(), preds.detach().numpy())
        self.log("r2", r2)
        self.log("rmse", rmse)
        self.log("loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        preds = self(
            batch.x.float(), batch.edge_attr.float(), batch.edge_index, batch.batch
        ).squeeze()
        target = batch.y.float()
        val_r2 = r2_score(preds, target)
        val_rmse = root_mean_squared_error(target.numpy(), preds.detach().numpy())
        self.log("val_r2", val_r2)
        self.log("val_rmse", val_rmse)
    def test_step(self, batch, batch_idx):
        preds = self(
            batch.x.float(), batch.edge_attr.float(), batch.edge_index, batch.batch
        ).squeeze()
        target = batch.y.float()
        test_rmse = root_mean_squared_error(target.numpy(), preds.detach().numpy())
        self.log("test_rmse", test_rmse)
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.parameters(),
            lr=self.lr,
            weight_decay = self.weight_decay)
        scheduler = torch.optim.lr_scheduler.ExponentialLR(
            optimizer, gamma=self.gamma
        )
        return [optimizer], [{"scheduler": scheduler, "interval": "epoch"}]

In [ ]:
# Trainer
def train_model(params):
    lr = params["lr"]
    weight_decay = params["weight_decay"]
    num_layers = params["num_layers"]
    hidden_size = params["hidden_size"]
    dense_size = params["dense_size"]
    pool_rate = params["pool_rate"]
    gamma = params["gamma"]
    normalization = params["normalization"]
    print(params)
    filename = f"PNA-{num_layers}-{lr:.2f}-{hidden_size}-{dense_size}"
    L.seed_everything(42)
    model = GNN_L(GNN("PNA", num_layers=num_layers, hidden_size=hidden_size, dense_size=dense_size, pool_rate=pool_rate, normalization = normalization),
                  lr,
                  weight_decay, gamma)
    # early_stopping = EarlyStopping("val_rmse", patience=10, mode="min")
    lr_monitor = LearningRateMonitor(logging_interval="epoch")
    checkpoint_callback = ModelCheckpoint(filename="{epoch}-{loss:.2f}-{rmse:.2f}",
                                          monitor="val_rmse",
                                          save_top_k=2,
                                          mode="min")

    logger = CSVLogger(save_dir="logs", name=filename)
    trainer = L.Trainer(
        max_epochs=100,
        callbacks=[lr_monitor, checkpoint_callback],
        log_every_n_steps=5,
        logger=logger,
        deterministic=True,
        accumulate_grad_batches=1
    )
    trainer.fit(model=model, train_dataloaders=zinc_dataloader)
    result = trainer.test(model, test_dataloader)
    result = result[0]['test_rmse']
    return result


In [ ]:
test_params = {
    "lr": 0.001,
    "weight_decay": 0,
    "num_layers": 2,
    "hidden_size": 32,
    "dense_size": 64,
    "gamma": 0.995,
    "pool_rate": 0.5,
    "normalization": True
}
train_model(test_params)

In [ ]:
params = {
    "lr": [0.01, 0.005, 0.001],
    "weight_decay": [0.01, 0.001, 0.0001, 0.00001],
    "num_layers": [0, 1, 2],
    "hidden_size": [4, 8, 16, 32, 64],
    "dense_size": [4, 8, 16, 32, 64],
    "gamma": [1, 0.995, 0.9, 0.8, 0.5],
    "pool_rate": [0.25, 0.5, 0.75],
    "normalization": [True]
}

In [ ]:
conf_dict = dict(num_iteration=100, domain_size=50000)
tuner = Tuner(params, train_model, conf_dict)
results = tuner.minimize()

In [ ]:
# params = {
#     "lr": [0.01, 0.005, 0.001],
#     "weight_decay": [0.01, 0.001, 0.0001, 0.00001],
#     "num_layers": [0, 1, 2],
#     "hidden_size": [4, 8, 16, 32, 64],
#     "dense_size": [4, 8, 16, 32, 64],
#     "gamma": [1, 0.995, 0.9, 0.8, 0.5],
#     "pool_rate": [0.25, 0.5, 0.75],
#     "normalization": [True]
# }

# conf_dict = dict(num_iteration=100, domain_size=50000)
# tuner = Tuner(params, train_model, conf_dict)
# results = tuner.minimize()

NameError: name 'pd' is not defined